Lo primero que haremos será crear la base de datos

In [0]:
%scala
spark.sql("CREATE DATABASE datos_padron")
spark.sql("USE datos_padron")

res0: org.apache.spark.sql.DataFrame = []

Creamos la tabla padron_txt con los campos del fichero CSV y cargamos los datos

In [0]:
%scala
spark.sql("DROP TABLE IF EXISTS padron_txt")
spark.sql("""CREATE TABLE padron_txt(
  COD_DISTRITO INT,
  DESC_DISTRITO STRING,
  COD_DIST_BARRIO INT,
  DESC_BARRIO STRING,
  COD_BARRIO INT,
  COD_DIST_SECCION INT,
  COD_SECCION INT,
  COD_EDAD_INT INT,
  EspanolesHombres INT,
  EspanolesMujeres INT,
  ExtranjerosHombres INT,
  ExtranjerosMujeres INT)
  ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' WITH SERDEPROPERTIES (
  "input.regex" = '"(\\d*)"\;"\\s*([^\\s]*)\\s*"\;"(\\d*)"\;"\\s*([^\\s]*)\\s*"\;"(\\d*)"\;"(\\d*)"\;"(\\d*)"\;"(\\d*)"\;"(\\d*)"\;"(\\d*)"\;"(\\d*)"\;"(\\d*)"'
  ) STORED AS TEXTFILE TBLPROPERTIES (
  "skip.header.line.count" = "1"
  )""")
spark.sql("LOAD DATA INPATH '/FileStore/shared_uploads/giovanni.rodriguez@bosonit.com/Rango_Edades.csv' INTO TABLE datos_padron.padron_txt")

res7: org.apache.spark.sql.DataFrame = []

Modificamos los datos de valor null a 0

In [0]:
%scala
spark.sql("""CREATE TABLE padron_txt_2 AS
  SELECT
    COD_DISTRITO,
    DESC_DISTRITO,
    COD_DIST_BARRIO,
    DESC_BARRIO,
    COD_BARRIO,
    COD_DIST_SECCION,
    COD_SECCION,
    COD_EDAD_INT,
    NVL(EspanolesHombres, 0) AS EspanolesHombres,
    NVL(EspanolesMujeres, 0) AS EspanolesMujeres,
    NVL(ExtranjerosHombres, 0) AS ExtranjerosHombres,
    NVL(ExtranjerosMujeres, 0) AS ExtranjerosMujeres
FROM padron_txt""")

res9: org.apache.spark.sql.DataFrame = [num_affected_rows: bigint, num_inserted_rows: bigint]

Almacenamos la tabla en formato Parquet

In [0]:
%scala
spark.sql("""CREATE TABLE padron_parquet_2
  STORED AS parquet AS
  SELECT * FROM padron_txt_2""")

res14: org.apache.spark.sql.DataFrame = []

Calculamos el total de EspanolesHombres, EspanolesMujeres, ExtranjerosHombres y ExtranjerosMujeres agrupado por DESC_DISTRITO y DESC_BARRIO

In [0]:
%scala
spark.sql("""SELECT DESC_DISTRITO, DESC_BARRIO,
   SUM(EspanolesHombres) AS EspanolesHombres,
   SUM(EspanolesMujeres) AS EspanolesMujeres,
   SUM(ExtranjerosHombres) AS ExtranjerosHombres,
   SUM(ExtranjerosMujeres) AS ExtranjerosMujeres
FROM padron_parquet_2
GROUP BY DESC_DISTRITO, DESC_BARRIO""").show()

+-------------+--------------+----------------+----------------+------------------+------------------+
DESC_DISTRITO| DESC_BARRIO|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+-------------+--------------+----------------+----------------+------------------+------------------+
 SALAMANCA| CASTELLANA| 6046| 7578| 1424| 2000|
 HORTALEZA| VALDEFUENTES| 29749| 30209| 3213| 4086|
 MORATALAZ| MARROQUINA| 11328| 13188| 845| 1053|
 CENTRO| JUSTICIA| 6937| 6591| 2340| 2295|
 CENTRO| UNIVERSIDAD| 12248| 12376| 4197| 4234|
 TETUAN| BERRUGUETE| 8649| 10312| 2777| 3588|
 CHAMARTIN| CASTILLA| 7146| 8183| 775| 997|
 ARGANZUELA| LEGAZPI| 8839| 9176| 767| 826|
 CHAMARTIN|HISPANOAMERICA| 13193| 15819| 1051| 1513|
 CENTRO| EMBAJADORES| 16340| 16283| 7983| 5945|
 CARABANCHEL| ABRANTES| 11152| 12960| 3379| 3806|
 BARAJAS| CORRALEJOS| 3534| 3588| 233| 291|
 CHAMBERI| VALLEHERMOSO| 8073| 10330| 597| 951|
 BARAJAS| TIMON| 5093| 5630| 564| 692|
 CHAMBERI| ARAPILES| 9439| 11569| 1435| 1729|
 null| null| 0| 0| 0| 0|
 MORATALAZ| PAVONES| 3545| 4285| 446| 492|
 SALAMANCA| RECOLETOS| 5693| 6751| 1352| 1821|
 CHAMARTIN| PROSPERIDAD| 14400| 17689| 1772| 2278|
 HORTALEZA| PALOMAS| 3054| 3131| 355| 403|
+-------------+--------------+----------------+----------------+------------------+------------------+
only showing top 20 rows

Creamos la tabla padron_particionado

In [0]:
%scala
spark.sql("DROP TABLE IF EXISTS padron_particionado")
spark.sql("""CREATE TABLE padron_particionado(
  COD_DISTRITO INT,
  COD_DIST_BARRIO INT,
  COD_BARRIO INT,
  COD_DIST_SECCION INT,
  COD_SECCION INT,
  COD_EDAD_INT INT,
  EspanolesHombres INT,
  EspanolesMujeres INT,
  ExtranjerosHombres INT,
  ExtranjerosMujeres INT)
PARTITIONED BY (DESC_DISTRITO STRING, DESC_BARRIO STRING)
STORED AS PARQUET""")

res26: org.apache.spark.sql.DataFrame = []

Insertamos los datos en ella

In [0]:
%scala
spark.sql("""INSERT INTO TABLE padron_particionado PARTITION(DESC_DISTRITO, DESC_BARRIO)
  SELECT
    COD_DISTRITO,
    COD_DIST_BARRIO,
    COD_BARRIO,
    COD_DIST_SECCION,
    COD_SECCION,
    COD_EDAD_INT,
    EspanolesHombres,
    EspanolesMujeres,
    ExtranjerosHombres,
    ExtranjerosMujeres,
    DESC_DISTRITO,
    DESC_BARRIO
  FROM padron_parquet_2""")

res32: org.apache.spark.sql.DataFrame = []

Calculamos la el total de EspanolesHombres, EspanolesMujeres, ExtranjerosHombres y ExtranjerosMujeres agrupado por DESC_DISTRITO y DESC_BARRIO para los distritos CENTRO, LATINA, CHAMARTIN, TETUAN, VICALVARO y BARAJAS

In [0]:
%scala
spark.sql("""SELECT
  DESC_DISTRITO, 
  DESC_BARRIO, 
  SUM(EspanolesHombres) AS EspanolesHombres,
  SUM(EspanolesMujeres) AS EspanolesMujeres,
  SUM(ExtranjerosHombres) AS ExtranjerosHombres,
  SUM(ExtranjerosMujeres) AS ExtranjerosMujeres
FROM padron_particionado
WHERE DESC_DISTRITO IN ('CENTRO', 'LATINA', 'CHAMARTIN', 'TETUAN', 'VICALVARO', 'BARAJAS') 
GROUP BY DESC_DISTRITO, DESC_BARRIO
ORDER BY DESC_DISTRITO""").show()

+-------------+--------------+----------------+----------------+------------------+------------------+
DESC_DISTRITO| DESC_BARRIO|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+-------------+--------------+----------------+----------------+------------------+------------------+
 BARAJAS| AEROPUERTO| 757| 698| 210| 231|
 BARAJAS| TIMON| 5093| 5630| 564| 692|
 BARAJAS| CORRALEJOS| 3534| 3588| 233| 291|
 CENTRO| UNIVERSIDAD| 12248| 12376| 4197| 4234|
 CENTRO| PALACIO| 9205| 9462| 2466| 2487|
 CENTRO| JUSTICIA| 6937| 6591| 2340| 2295|
 CENTRO| CORTES| 3925| 3923| 1438| 1600|
 CENTRO| SOL| 2855| 2548| 1445| 1372|
 CENTRO| EMBAJADORES| 16340| 16283| 7983| 5945|
 CHAMARTIN| CASTILLA| 7146| 8183| 775| 997|
 CHAMARTIN| PROSPERIDAD| 14400| 17689| 1772| 2278|
 CHAMARTIN|HISPANOAMERICA| 13193| 15819| 1051| 1513|
 LATINA| ALUCHE| 24808| 29563| 5586| 6546|
 LATINA| CAMPAMENTO| 7095| 8095| 1980| 2441|
 LATINA| LUCERO| 13824| 15700| 3271| 3701|
 TETUAN| ALMENARA| 8732| 10182| 1750| 2188|
 TETUAN| BERRUGUETE| 8649| 10312| 2777| 3588|
 TETUAN| CASTILLEJOS| 7808| 9425| 1385| 1902|
 TETUAN| VALDEACEDERAS| 9478| 10548| 2850| 3439|
 VICALVARO| VALDEBERNARDO| 7614| 8407| 754| 767|
+-------------+--------------+----------------+----------------+------------------+------------------+
only showing top 20 rows

Algunas consultas de agregación

In [0]:
%scala
spark.sql("""SELECT DISTINCT
  DESC_BARRIO,
  COUNT(DESC_BARRIO) AS numBarrios,
  AVG(EspanolesHombres) AS AvgEspanolesHombres,
  MAX(EspanolesMujeres) AS maxEspanolesMujeres,
  MIN(ExtranjerosHombres) AS minExtranjerosHombres
FROM padron_particionado
GROUP BY DESC_DISTRITO, DESC_BARRIO
""").show()

+--------------+----------+-------------------+-------------------+---------------------+
 DESC_BARRIO|numBarrios|AvgEspanolesHombres|maxEspanolesMujeres|minExtranjerosHombres|
+--------------+----------+-------------------+-------------------+---------------------+
 ARAPILES| 2183| 4.323866239120476| 21| 0|
 OPA�EL| 2554| 4.560689115113547| 35| 0|
 VALDEZARZA| 2345| 5.188059701492537| 35| 0|
 PALACIO| 1767| 5.2093944538766275| 20| 0|
 TRAFALGAR| 2267| 4.126599029554478| 19| 0|
 PROSPERIDAD| 2669| 5.395279130760584| 25| 0|
 GOYA| 2656| 4.071536144578313| 18| 0|
 ABRANTES| 2031| 5.490891186607582| 45| 0|
 LUCERO| 2851| 4.848824973693441| 41| 0|
 ACACIAS| 2638| 5.826004548900682| 35| 0|
 VALDEFUENTES| 2545| 11.689194499017681| 127| 0|
 EMBAJADORES| 3485| 4.6886657101865135| 22| 0|
 MARROQUINA| 2408| 4.704318936877076| 47| 0|
 BUENAVISTA| 2768| 7.17485549132948| 66| 0|
 ALUCHE| 5750| 4.3144347826086955| 30| 0|
 UNIVERSIDAD| 2620| 4.674809160305344| 19| 0|
 CANILLAS| 3021| 5.536577292287322| 34| 0|
HISPANOAMERICA| 2106| 6.264482431149098| 27| 0|
 ARGUELLES| 1988| 4.747987927565393| 21| 0|
 GUINDALERA| 3364| 4.9613555291319855| 27| 0|
+--------------+----------+-------------------+-------------------+---------------------+
only showing top 20 rows